In [1]:
import pandas as pd
import numpy as np
import io

In [2]:
# Read the Excel file into a DataFrame
df = pd.read_excel(r"/home/samyak/PycharmProjects/Python_Learn /Decision Tree Model/DataSet/PSP_Jan_Feb_2019.xlsx")

# Display the DataFrame
print(df.head())

   Unnamed: 0                tmsp  country  amount  success         PSP  \
0           0 2019-01-01 00:01:11  Germany      89        0     UK_Card   
1           1 2019-01-01 00:01:17  Germany      89        1     UK_Card   
2           2 2019-01-01 00:02:49  Germany     238        0     UK_Card   
3           3 2019-01-01 00:03:13  Germany     238        1     UK_Card   
4           4 2019-01-01 00:04:33  Austria     124        0  Simplecard   

   3D_secured    card  
0           0    Visa  
1           0    Visa  
2           1  Diners  
3           1  Diners  
4           0  Diners  


In [3]:
print(df.columns)

Index(['Unnamed: 0', 'tmsp', 'country', 'amount', 'success', 'PSP',
       '3D_secured', 'card'],
      dtype='object')


In [4]:
missing_values= df.isnull().sum()

In [5]:
missing_data =pd.DataFrame({
    'Missing Values': missing_values,
    #'Percentage Missing Value': Percentage_missing_value
})
print(f"The dataframe has {len(missing_values)} rows with missing values.")

The dataframe has 8 rows with missing values.
